In [1]:
import numpy as np
def get_words_embeding(glove_file):
    words_embeding_dict = {}
    words_embeding_set = set()#别人训练的glove所包含的词集合
    for word_vector in open(glove_file,"r",encoding='utf-8').readlines():
        word_vector = word_vector.rstrip().split()
        #print(word_vector)
        word = word_vector[0]  #这个零 代表的是索引  下面的 一  也是同样的道理
        vector = np.asarray(word_vector[1:], "float32")
        words_embeding_dict[word] = vector
        words_embeding_set.add(word)
    return words_embeding_dict,words_embeding_set
#返回的一个为word和对应的嵌入的字典，另一个为glove别人训练的模型中包含的word
words_embeding_dict,words_embeding_set = get_words_embeding(r"/home/kayzhou/zhangyue/text_ML/data/glove.twitter.27B.200d.txt")

In [3]:
from tqdm import tqdm
# classfied_hashtag_list = []
# for line in open(r"E:\hashtag标注\top200.txt","r"):
#     classfied_hashtag = line.strip().split(" ")[0]
#     classfied_hashtag_list.append(classfied_hashtag)

tweet_embedding = []
tag_list = []
lujing = [r"/home/kayzhou/zhangyue/text_ML/data/one_tweet_class_train.txt",
         r"/home/kayzhou/zhangyue/text_ML/data/one_tweet_class_dev.txt"]
j=0
for i in lujing:
    for line in tqdm(open(i)):
        line = line.strip().split(" ") 
#         print(line)
        text = line[0:len(line)-1]
#         print(text)
        tag = line[-1]

        one_tweet_embeding = np.array([0]*200,dtype="float64")

        embeding_tweet_len = 0
        j=j+1
        for word in text:
            if word in words_embeding_set: 
                one_tweet_embeding += np.array(words_embeding_dict[word])  
                embeding_tweet_len += 1
        if embeding_tweet_len==0:
            one_tweet_final_embeding = one_tweet_embeding
        else:
            one_tweet_final_embeding = one_tweet_embeding/embeding_tweet_len
        tweet_embedding.append(one_tweet_final_embeding) 
        tag_list.append(int(tag))

3760566it [04:36, 13609.45it/s]
417841it [00:30, 13706.85it/s]


In [4]:
test_tweet_embedding = []
test_tag_list = []

for line in tqdm(open(r"/home/kayzhou/zhangyue/text_ML/data/one_tweet_class_test.txt")):
    line = line.strip().split(" ") 
#         print(line)
    text = line[0:len(line)-1]
#         print(text)
    tag = line[-1]

    one_tweet_embeding = np.array([0]*200,dtype="float64")

    embeding_tweet_len = 0
    for word in text:
        if word in words_embeding_set: 
            one_tweet_embeding += np.array(words_embeding_dict[word])  
            embeding_tweet_len += 1
    if embeding_tweet_len==0:
        one_tweet_final_embeding = one_tweet_embeding
    else:
        one_tweet_final_embeding = one_tweet_embeding/embeding_tweet_len
    test_tweet_embedding.append(one_tweet_final_embeding) 
    test_tag_list.append(int(tag))


4000it [00:00, 13924.12it/s]


# 单条推文机器学习

In [5]:
from sklearn.model_selection import train_test_split
import joblib
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC

In [6]:
# Multinomial Naive Bayes Classifier
def naive_bayes_classifier(train_x, train_y):
    #连续性数据
    from sklearn.naive_bayes import GaussianNB
    model = GaussianNB()
    model.fit(train_x, train_y)
    return model


# KNN Classifier
def knn_classifier(train_x, train_y):
    from sklearn.neighbors import KNeighborsClassifier
    model = KNeighborsClassifier()
    model.fit(train_x, train_y)
    return model


# Logistic Regression Classifier
def logistic_regression_classifier(train_x, train_y):
    from sklearn.linear_model import LogisticRegression
    model = LogisticRegression(penalty='l2')
    model.fit(train_x, train_y)
    return model


# Random Forest Classifier
def random_forest_classifier(train_x, train_y):
    from sklearn.ensemble import RandomForestClassifier
    model = RandomForestClassifier(n_estimators=8)
    model.fit(train_x, train_y)
    return model


# Decision Tree Classifier
def decision_tree_classifier(train_x, train_y):
    from sklearn import tree
    model = tree.DecisionTreeClassifier()
    model.fit(train_x, train_y)
    return model


# GBDT(Gradient Boosting Decision Tree) Classifier
def gradient_boosting_classifier(train_x, train_y):
    from sklearn.ensemble import GradientBoostingClassifier
    model = GradientBoostingClassifier(n_estimators=200)
    model.fit(train_x, train_y)
    return model


# SVM Classifier
def svm_classifier(train_x, train_y):
    from sklearn.svm import SVC
    model = SVC(kernel='rbf', probability=True)
    model.fit(train_x, train_y)
    return model


# SVM Linear Classifier
def svm_linear_classifier(train_x, train_y):
    model = LinearSVC()
    model.fit(train_x, train_y)
    return model


# SVM Classifier using cross validation
def svm_cross_validation(train_x, train_y):
    from sklearn.model_selection import GridSearchCV
    from sklearn.svm import SVC
    model = SVC(kernel='rbf', probability=True)
    #1e-3, 1e-2, 1e-1, 1, 10,
    param_grid = {'C': [100, 1000], 'gamma': [0.001, 0.0001]}
    grid_search = GridSearchCV(model, param_grid, n_jobs=1, verbose=1)
    grid_search.fit(train_x, train_y)
    best_parameters = grid_search.best_estimator_.get_params()
    for para, val in list(best_parameters.items()):
        print(para, val)
    model = SVC(kernel='rbf', C=best_parameters['C'],
                gamma=best_parameters['gamma'], probability=True)
    model.fit(train_x, train_y)

In [7]:
def train():
    # read data
#     X, y = tweet_embeding, class_list

#     print("Reading data finished! count:", len(y))

    # split train and test data
    X_train, X_test, y_train, y_test = tweet_embedding,test_tweet_embedding,tag_list,test_tag_list

    print("Splitting data finished!")

#     # build one hot embedding
#     # v = DictVectorizer(dtype=np.int8, sparse=True, sort=False)
#     v = TfidfVectorizer(ngram_range=(1, 2), max_features=1000000)
#     X_train = v.fit_transform(X_train)
#     X_test = v.transform(X_test)

#     # joblib.dump(v, f'data/{self.train_dir}/DictVectorizer.joblib')
#     joblib.dump(v, f'data/{self.train_dir}/TfidfVectorizer.joblib')
#     print("Building word embedding finished!")
#     print(X_train[0].shape, X_train[1].shape)
#     print(X_train.shape, X_test.shape)

    # X_train, y_train = SMOTE().fit_sample(X_train, y_train)
    # X_train, y_train = ADASYN().fit_sample(X_train, y_train)
    # X_train, y_train = RandomOverSampler(random_state=24).fit_sample(X_train, y_train)
    # X_train, y_train = RandomUnderSampler(random_state=42).fit_sample(X_train, y_train)

    print("After sampling!")
#     print(X_train.shape, X_test.shape)

    # machine learning model
    
    #"LR","NB","KNN","RF","DT","GBDT","SVM" ,"SVMCV","SVC"
    list_classifiers = ['LR','NB','DT','RF',"GBDT"]
    # list_classifiers = ['SVC']
    # list_classifiers = ['GBDT']

    classifiers = {
            'NB': naive_bayes_classifier,            
            'KNN': knn_classifier,
            'LR': logistic_regression_classifier,
            'RF': random_forest_classifier,
            'DT': decision_tree_classifier,
            'SVM': svm_classifier,
            'SVMCV': svm_cross_validation,
            'GBDT': gradient_boosting_classifier,
            'SVC': svm_linear_classifier,
    }

    for classifier in list_classifiers:
        print('******************* {} ********************'.format(classifier))
        if classifier == "LR":
            # clf = LogisticRegression(penalty='l2', multi_class="multinomial", solver="sag", max_iter=10e8)
            clf = LogisticRegression(penalty='l2', solver="sag", max_iter=10e8, multi_class="auto")
            clf.fit(X_train, y_train)
        elif classifier == "GBDT":
            clf = GradientBoostingClassifier(
                learning_rate=0.1, max_depth=3)
            clf.fit(X_train, y_train)
        else:
            clf = classifiers[classifier](X_train, y_train)
        print("fitting finished! Lets evaluate!")
        evaluate(clf, X_train, y_train, X_test, y_test)

In [8]:
def evaluate(clf, X_train, y_train, X_test, y_test):
    # CV
    # print('accuracy of CV=10:', cross_val_score(
    #     clf, X_train, y_train, cv=10).mean())

    y_pred = clf.predict(X_test)
    print(classification_report(y_test, y_pred))

In [9]:
train()

Splitting data finished!
After sampling!
******************* LR ********************
fitting finished! Lets evaluate!
              precision    recall  f1-score   support

           0       0.59      0.85      0.70      2008
           1       0.73      0.41      0.52      1992

    accuracy                           0.63      4000
   macro avg       0.66      0.63      0.61      4000
weighted avg       0.66      0.63      0.61      4000

******************* NB ********************
fitting finished! Lets evaluate!
              precision    recall  f1-score   support

           0       0.55      0.80      0.65      2008
           1       0.62      0.34      0.44      1992

    accuracy                           0.57      4000
   macro avg       0.59      0.57      0.54      4000
weighted avg       0.58      0.57      0.54      4000

******************* DT ********************
fitting finished! Lets evaluate!
              precision    recall  f1-score   support

           0       